<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>DiaObject Low-z SNIa Sample Identification</b> <br>
Contact author: Melissa Graham <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: medium <br>
Targeted learning level: intermediate <br>

In [ ]:
%load_ext pycodestyle_magic
%flake8_on
import logging
logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** To use the `DiaObject` catalog parameters to identify potential Type Ia supernovae at low redshifts, $0.1<z<0.4$.

**Skills:** TAP service. `DiaObject` use.

**LSST Data Products:** _List the all of the LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:** Originally developed by Melissa Graham and the Rubin Community Engagement Team for Data Preview 0. Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._


 * numpy
 * matplotlib
 * astropy.cosmology -- https://docs.astropy.org/en/stable/cosmology/index.html
 * lsst -- pipelines.lsst.io

In [ ]:
# general python packages
import numpy
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
# import lsst.daf.butler as dafButler

# LSST package for image display
# import lsst.afw.display as afwDisplay

# astropy package for cosmology
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

In [ ]:
service = get_tap_service()

In [ ]:
redshift_min = 0.1
redshift_max = 0.4

lumdist_min = cosmo.luminosity_distance(redshift_min).value
lumdist_max = cosmo.luminosity_distance(redshift_max).value

distmod_min = 5.0 * numpy.log10(lumdist_min) + 25.0
distmod_max = 5.0 * numpy.log10(lumdist_max) + 25.0

print('min: ', redshift_min, lumdist_min, distmod_min)
print('max: ', redshift_max, lumdist_max, distmod_max)

In [ ]:
snia_peak_Mg = -19.0
snia_peak_Mr = -19.0
snia_peak_Mi = -19.0
snia_peak_deltamag = 0.5

In [ ]:
snia_peak_mg_min = distmod_min + snia_peak_Mg - snia_peak_deltamag
snia_peak_mg_max = distmod_max + snia_peak_Mg + snia_peak_deltamag
snia_peak_mr_min = distmod_min + snia_peak_Mr - snia_peak_deltamag
snia_peak_mr_max = distmod_max + snia_peak_Mr + snia_peak_deltamag
snia_peak_mi_min = distmod_min + snia_peak_Mi - snia_peak_deltamag
snia_peak_mi_max = distmod_max + snia_peak_Mi + snia_peak_deltamag

print('snia_peak_mg_min = ', snia_peak_mg_min)
print('snia_peak_mg_max = ', snia_peak_mg_max)
print('snia_peak_mr_min = ', snia_peak_mr_min)
print('snia_peak_mr_max = ', snia_peak_mr_max)
print('snia_peak_mi_min = ', snia_peak_mi_min)
print('snia_peak_mi_max = ', snia_peak_mi_max)

## 2. Section Heading

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

Find DiaObjects that have peak magnitudes consistent with a Type Ia supernova.

In [ ]:
results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_fluxToAbMag(gPSFluxMax/1e32) AS gMagMin, "
                         "scisql_fluxToAbMag(gPSFluxMin/1e32) AS gMagMax, "
                         "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rMagMin, "
                         "scisql_fluxToAbMag(rPSFluxMin/1e32) AS rMagMax, "
                         "scisql_fluxToAbMag(iPSFluxMax/1e32) AS iMagMin, "
                         "scisql_fluxToAbMag(iPSFluxMin/1e32) AS iMagMax  "
                         "FROM dp02_test_PREOPS863_00.DiaObject "
                         "WHERE scisql_fluxToAbMag(gPSFluxMax/1e32) > "+str(snia_peak_mg_min)+" "
                         "AND scisql_fluxToAbMag(gPSFluxMax/1e32) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_fluxToAbMag(rPSFluxMax/1e32) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_fluxToAbMag(rPSFluxMax/1e32) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_fluxToAbMag(iPSFluxMax/1e32) > "+str(snia_peak_mi_min)+" "
                         "AND scisql_fluxToAbMag(iPSFluxMax/1e32) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_fluxToAbMag(gPSFluxMin/1e32) - scisql_fluxToAbMag(gPSFluxMax/1e32) > 0.5 "
                         "AND scisql_fluxToAbMag(rPSFluxMin/1e32) - scisql_fluxToAbMag(rPSFluxMax/1e32) > 0.5 "
                         "AND scisql_fluxToAbMag(iPSFluxMin/1e32) - scisql_fluxToAbMag(iPSFluxMax/1e32) > 0.5 "
                         "AND CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 10.0)) = 1 ",
                         maxrec=1000)
DiaObjs = results.to_table()
del results

In [ ]:
len(DiaObjs)

In [ ]:
DiaObjs

Get their lightcurves from DiaSource catalog. Are they SNIa-like?

In [ ]:
all_filters = ['u','g','r','i','z','y']
all_filtclr = ['darkviolet','darkgreen','red','darkorange','brown','black']

fig, ax = plt.subplots( len(DiaObjs), figsize=(14,14), sharey=False, sharex=False )

for j in range(len(DiaObjs)):
    sel_diaObjectId = DiaObjs['diaObjectId'][j]
    results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, filterName, midPointTai, psFlux, psFluxErr "
                             "FROM dp02_test_PREOPS863_00.DiaSource "
                             "WHERE diaObjectId = "+str(sel_diaObjectId))
    results = results.to_table()
    results['psAbMag'] = -2.50 * numpy.log10(results['psFlux']) + 31.4
    
    for f,filt in enumerate(all_filters):
        fx = numpy.where( results['filterName'] == filt )[0]
        ax[j].plot( results['midPointTai'][fx], results['psAbMag'][fx], 'o', ms=5, mew=0, alpha=0.8, color=all_filtclr[f] )
    
    ax[j].set_ylim([numpy.max(results['psAbMag'])+0.3, numpy.min(results['psAbMag'])-0.3])
    
    del results